<a href="https://colab.research.google.com/github/hadwin-357/ML_models/blob/main/Naive_Baynes_Classifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Naive Baynes Classifer
five steps
Step 1: Separate By Class.
Step 2: Summarize Dataset.
Step 3: Summarize Data By Class.
Step 4: Gaussian Probability Density Function.
Step 5: Class Probabilities.

In [4]:
import requests
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
response= requests.get(url)
with open ('./irs.csv', 'wb') as f:
  f.write(response.content)

In [5]:
import pandas as pd

'''
This is the data
Sepal length in cm.
Sepal width in cm.
Petal length in cm.
Petal width in cm.
Class
'''
columns=['S_length','S_width','P_length','P_width','Class']
df= pd.read_csv('./irs.csv',names=columns)

df.sample(5, random_state=42)
#df.head(5)

,S_length,S_width,P_length,P_width,Class
73,6.1,2.8,4.7,1.2,Iris-versicolor
18,5.7,3.8,1.7,0.3,Iris-setosa
118,7.7,2.6,6.9,2.3,Iris-virginica
78,6.0,2.9,4.5,1.5,Iris-versicolor
76,6.8,2.8,4.8,1.4,Iris-versicolor


In [6]:
# data preprocessing

classes=set(df.Class)

hash_lookup={}
#make a hash table for class
for i, value in enumerate(classes):
  hash_lookup[value]=i

df['ID']=df['Class'].apply(lambda x: hash_lookup[x])
df=df.drop(['Class'], axis=1)
df.sample(5, random_state=42)

,S_length,S_width,P_length,P_width,ID
73,6.1,2.8,4.7,1.2,2
18,5.7,3.8,1.7,0.3,0
118,7.7,2.6,6.9,2.3,1
78,6.0,2.9,4.5,1.5,2
76,6.8,2.8,4.8,1.4,2


In [7]:
# #step 1: seperate data by class
# #print(classes)
# data_sep={}
# for item in classes:
# data_sep[item]=df[df['Class']==item]
# data_sep['Iris-setosa'].head(5)

In [140]:
#step 2. summarize data
import numpy as np
def summarize_dataset(dataset):
  summaries = [(np.mean(dataset[column]), np.std(dataset[column]), len(dataset[column])) for column in dataset.columns]
  return summaries
#df=df.drop(['Class'],axis=1)

sum_overall = summarize_dataset(df)


In [9]:
sum_overall

[(5.843333333333334, 0.8253012917851409, 150),
 (3.0540000000000003, 0.4321465800705435, 150),
 (3.758666666666666, 1.7585291834055212, 150),
 (1.1986666666666668, 0.7606126185881716, 150),
 (1.0, 0.816496580927726, 150)]

In [75]:
np.std(df.values,axis=0)

array([0.82530129, 0.43214658, 1.75852918, 0.76061262, 0.81649658])

In [141]:
#step 3. summarize categorized data

sum_subclass={}
for item in classes:
  sum_subclass[item]= summarize_dataset(df[df['ID']==hash_lookup[item]])
sum_subclass

{'Iris-setosa': [(5.006, 0.3489469873777391, 50),
  (3.418, 0.37719490982779713, 50),
  (1.464, 0.17176728442867112, 50),
  (0.244, 0.10613199329137281, 50),
  (0.0, 0.0, 50)],
 'Iris-virginica': [(6.587999999999998, 0.6294886813914926, 50),
  (2.974, 0.3192553836664309, 50),
  (5.5520000000000005, 0.546347874526844, 50),
  (2.0260000000000002, 0.2718896835115301, 50),
  (1.0, 0.0, 50)],
 'Iris-versicolor': [(5.936, 0.5109833656783751, 50),
  (2.7700000000000005, 0.31064449134018135, 50),
  (4.26, 0.4651881339845203, 50),
  (1.3259999999999998, 0.19576516544063705, 50),
  (2.0, 0.0, 50)]}

In [142]:
#step four generate probalities distribution for each features
# assume the distribution is guassian distribution
# Example of Gaussian PDF
from math import sqrt
from math import pi
from math import exp

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
 exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
 return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [146]:
#step 5 calculate class probability
#P(class|data) = P(X|class) * P(class)
#P(class) is easy to calculate ; Based on multimode assumption: P(X|class)= p(x1|class)*p(x2|class)*....
#check course material:https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote05.html
def calculate_class_probablity(sum_subclass,data):
  '''
  This function is to calculate class probablity
  args:
  sum_subclass: dictionary, containing mean std in differnt classes
  data: a row in df for input to predict class
  '''
  probability={} # to hold results
  total_no_data = np.sum([sum_subclass[label][0][2] for label in sum_subclass.keys()])
  for index, (key, value) in enumerate(sum_subclass.items()):
    probability[key]= value[0][2]/total_no_data
    for i in range(len(value)-1): # -1 to remove statistics of label
      probability[key] *= calculate_probability(data[i],  value[i][0], value[i][1])

  return probability



In [149]:
probabilities = calculate_class_probablity(sum_subclass, df.iloc[1].values[:-1])

In [152]:
np.array(list(probabilities.values()))

array([1.55679060e+00, 3.57662796e-25, 2.30725683e-17])

In [104]:
data_set=df.values

def summarize_dataset(dataset):
  mean=np.mean(dataset,axis=0)
  std = np.std(dataset,axis=0)
  summaries = [mean, std, dataset.shape[0]]
  return summaries

def summarize_by_class(dataset:np.array):
  sum_subclass={}
  for item in classes:
    sum_subclass[item]= summarize_dataset(dataset[dataset[:,-1]==hash_lookup[item]])
  return(sum_subclass)

summ=summarize_by_class(data_set)
def calculate_class_probablity(sum_subclass,test):
  '''
  This function is to calculate class probablity
  args:
  sum_subclass: dictionary, containing mean std in differnt classes
  data: a row in df for input to predict class
  '''
  probability={} # to hold results
  total_no_data = np.sum([sum_subclass[label][2] for label in sum_subclass.keys()])
  for index, (key, value) in enumerate(sum_subclass.items()):
    probability[key]= value[2]/total_no_data
    for i in range(len(value)-1): # -1 to remove statistics of label
      probability[key] *= calculate_probability(test[i],  value[0][i], value[1][i])

  return probability

prob=calculate_class_probablity(sum, X_test[2])


In [116]:
prob=calculate_class_probablity(sum, X_test[3])
prob

{'Iris-setosa': 0.0027154243376214054,
 'Iris-virginica': 0.16612668700372013,
 'Iris-versicolor': 0.3038037447075301}

In [155]:
# final train and test; instead of pd dataframe, simply use df.values()( numpy.array) as the input data
#helper function
# Calculate accuracy percentage
import random
import numpy as np

def accuracy_metric(actual, predicted):
 return (actual==predicted).sum() / float(len(actual)) * 100.0

# Predict the class for a given row
def predict(summaries, row):
  probabilities = calculate_class_probablity(summaries, row)
  predictID = np.array(list(probabilities.values())).argmax()
  return predictID


def summarize_dataset(dataset):
  mean=np.mean(dataset,axis=0)
  std = np.std(dataset,axis=0)
  summaries = [mean, std, dataset.shape[0]]
  return summaries

def summarize_by_class(data:np.array):
  sum_subclass={}
  for item in classes:
    sum_subclass[item]= summarize_dataset(data[data[:,-1]==hash_lookup[item]])
  return(sum_subclass)

def calculate_class_probablity(sum_subclass,test):
  '''
  This function is to calculate class probablity
  args:
  sum_subclass: dictionary, containing mean std in differnt classes
  data: a row in df for input to predict class
  '''
  probability={} # to hold results
  total_no_data = np.sum([sum_subclass[label][2] for label in sum_subclass.keys()])
  for index, (key, value) in enumerate(sum_subclass.items()):
    probability[key]= value[2]/total_no_data
    for i in range(len(value)-1): # -1 to remove statistics of label
      probability[key] *= calculate_probability(test[i],  value[0][i], value[1][i])

  return probability


 # Naive Bayes Algorithm
def naive_bayes(train, test):
 summarize = summarize_by_class(train)
 predictions = list()
 for i in range(test.shape[0]):
  output = predict(summarize, test[i])
  predictions.append(output)
 return(predictions)



In [118]:
from sklearn.model_selection import train_test_split

X = df.copy().values # containing label for subclass seperation
y=df['ID'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [158]:
preds=naive_bayes(X_train,X_test)


In [159]:
acc =accuracy_metric(y_test, preds)
acc

82.22222222222221